In [1]:
from logger import Logger

import pandas as pd
import numpy as np
import os

In [2]:
logger = Logger(show = True, html_output = True, config_file = "config.txt")

logger.log("Reading transactions file...")
df = pd.read_csv(logger.get_data_file(logger.config_dict['DATA_FILE']))
logger.log("Finish reading {} rows".format(df.shape[0]), show_time = True)
print(os.linesep + "Data snippet")
df.head()

2019.05.08-01:16:09 Initialize the logger
2019.05.08-01:16:09 Create logs folder logs
2019.05.08-01:16:09 Create log file logs\2019-05-08_01_16_09_log0.html
2019.05.08-01:16:09 Create models folder models
2019.05.08-01:16:09 Create output folder output
2019.05.08-01:16:09 Create data folder data
2019.05.08-01:16:09 Read config file config.txt
[2019.05.08-01:16:09] Reading transactions file...
[2019.05.08-01:16:10] Finish reading 290382 rows [0.93s]

Data snippet


,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id
0,1,2015-11-09 14:26:51,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-01 23:03:11,MexicoAccount,email68370,ip111778,card184798
1,2,2015-11-09 14:27:38,MX,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-02 04:50:55,MexicoAccount,email101299,ip78749,card151595
2,3,2015-11-23 16:34:16,MX,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-02 14:30:28,MexicoAccount,email278604,ip70594,card242142
3,4,2015-11-23 16:34:51,MX,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-03 07:53:37,MexicoAccount,email47409,ip113648,card181744
4,5,2015-11-09 14:26:08,MX,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-08 18:35:35,MexicoAccount,email205501,ip83553,card97271


In [3]:
crt_size = df.shape[0]

df['mail_id'] = df['mail_id'].str.replace('email','')
df.drop(df[df.mail_id == "NA"].index, inplace=True)
df['mail_id'] = pd.to_numeric(df['mail_id'])
logger.log("Dropping NA in email id {}".format(crt_size - df.shape[0]))
crt_size = df.shape[0]

df['ip_id']   = df['ip_id'].str.replace('ip','')
df.drop(df[df.ip_id == "NA"].index, inplace=True)
df['ip_id'] = pd.to_numeric(df['ip_id'])
logger.log("Dropping NA in ip id {}".format(crt_size - df.shape[0]))
crt_size = df.shape[0]

df['card_id'] = df['card_id'].str.replace('card','')
df.drop(df[df.card_id == "NA"].index, inplace=True)
df['card_id'] = pd.to_numeric(df['card_id'])
logger.log("Dropping NA in card id {}".format(crt_size - df.shape[0]))
crt_size = df.shape[0]

[2019.05.08-01:16:10] Dropping NA in email id 366
[2019.05.08-01:16:11] Dropping NA in ip id 0
[2019.05.08-01:16:11] Dropping NA in card id 0


In [4]:
df.drop(df[df.simple_journal == "Refused"].index, inplace=True)
logger.log("Dropping REFUSED transaction {}".format(crt_size - df.shape[0]))
crt_size = df.shape[0] 

[2019.05.08-01:16:12] Dropping REFUSED transaction 53318


In [5]:
logger.log("Number of NaNs per column:" + os.linesep + "{}".format(df.isna().sum()))

[2019.05.08-01:16:12] Number of NaNs per column:
txid                                0
bookingdate                         0
issuercountrycode                   4
txvariantcode                       0
bin                                 0
amount                              0
currencycode                        0
shoppercountrycode                342
shopperinteraction                  0
simple_journal                      0
cardverificationcodesupplied    13194
cvcresponsecode                     0
creationdate                        0
accountcode                         0
mail_id                             0
ip_id                               0
card_id                             0
dtype: int64


In [6]:
print(os.linesep + "Few examples of NaNs rows:")
df[df.isna().any(axis=1)].head()


Few examples of NaNs rows:


,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id
25297,39506,2015-09-27 20:21:39,MX,mccredit,528843.0,61530.0,MXN,NaN,Ecommerce,Settled,True,0,2015-09-27 20:21:27,MexicoAccount,263004.0,86356.0,39750.0
25592,40058,2015-09-30 06:32:50,NaN,mc,520116.0,119940.0,MXN,MX,Ecommerce,Settled,True,0,2015-09-30 06:32:40,MexicoAccount,47407.0,61517.0,46742.0
27227,43731,2015-10-19 20:40:32,NaN,mc,520116.0,44800.0,MXN,MX,Ecommerce,Settled,True,0,2015-10-19 20:40:16,MexicoAccount,47407.0,61517.0,46742.0
27307,43927,2015-10-20 20:30:51,NaN,mc,520116.0,111920.0,MXN,MX,Ecommerce,Settled,True,0,2015-10-20 20:30:43,MexicoAccount,164167.0,76472.0,269591.0
31536,50220,2015-07-02 04:35:15,AU,mccredit,535316.0,11000.0,AUD,AU,Ecommerce,Settled,NaN,0,2015-07-01 03:45:57,APACAccount,267790.0,17828.0,194191.0


In [7]:
logger.log("NaNs from issuercountrycode that are Chargeback: {}".format(
    (df[df['issuercountrycode'].isna()].simple_journal == "Chargeback").sum()))
logger.log("NaNs from shoppercountrycode that are Chargeback: {}".format(
    (df[df['shoppercountrycode'].isna()].simple_journal == "Chargeback").sum()))
logger.log("NaNs from cardverificationcodesupplied that are Chargeback: {}".format(
    (df[df['cardverificationcodesupplied'].isna()].simple_journal == "Chargeback").sum()))

[2019.05.08-01:16:12] NaNs from issuercountrycode that are Chargeback: 0
[2019.05.08-01:16:12] NaNs from shoppercountrycode that are Chargeback: 0
[2019.05.08-01:16:12] NaNs from cardverificationcodesupplied that are Chargeback: 0


In [8]:
df.dropna(axis = 0, inplace = True)
logger.log("Drop the other NaNs: {} entries".format(crt_size - df.shape[0]))
crt_size = df.shape[0]

[2019.05.08-01:16:12] Drop the other NaNs: 13526 entries


In [9]:
logger.log("Convert datatypes for numeric and timestamps")

df['bookingdate']  = pd.to_datetime(df['bookingdate'])
df['creationdate'] = pd.to_datetime(df['creationdate'])
df = df.infer_objects()

df['bin'] = df['bin'].astype(object)
df['mail_id'] = df['mail_id'].astype(object)
df['card_id'] = df['card_id'].astype(object)
df['ip_id'] = df['ip_id'].astype(object)

print(os.linesep + "{}".format(df.dtypes))

[2019.05.08-01:16:12] Convert datatypes for numeric and timestamps

txid                                     int64
bookingdate                     datetime64[ns]
issuercountrycode                       object
txvariantcode                           object
bin                                     object
amount                                 float64
currencycode                            object
shoppercountrycode                      object
shopperinteraction                      object
simple_journal                          object
cardverificationcodesupplied              bool
cvcresponsecode                          int64
creationdate                    datetime64[ns]
accountcode                             object
mail_id                                 object
ip_id                                   object
card_id                                 object
dtype: object


In [10]:
logger.log("Sort after creationdate")
df.sort_values('creationdate', inplace = True)
df.reset_index(drop = True, inplace = True)

[2019.05.08-01:16:12] Sort after creationdate


In [11]:
logger.log("Currencies identified in data are {}".format(np.unique(df.currencycode.values)))

conversion = {'AUD': 0.699165, 'GBP': 1.31061, 'MXN': 0.222776586, 'NZD': 0.66152, 'SEK': 0.104405}
logger.log("Convert all amounts in USD using rates: {}".format(conversion))
df['amount'] = df.apply(lambda e: e['amount'] * conversion[e['currencycode']], axis=1)
logger.log("Done converting", show_time = True)

[2019.05.08-01:16:13] Currencies identified in data are ['AUD' 'GBP' 'MXN' 'NZD' 'SEK']
[2019.05.08-01:16:13] Convert all amounts in USD using rates: {'AUD': 0.699165, 'GBP': 1.31061, 'MXN': 0.222776586, 'NZD': 0.66152, 'SEK': 0.104405}
[2019.05.08-01:16:17] Done converting [4.71s]


In [12]:
logger.log("Drop txid and bookingdate")
df.drop(["txid", "bookingdate"], inplace = True, axis = 1)

logger.log("Change simple_journal to label")
df.replace({'simple_journal': {"Settled": 0, "Chargeback": 1}}, inplace = True)
df.rename(columns = {'simple_journal': 'label'}, inplace = True)

logger.log("Rearrange columns")
columns = ["creationdate", "card_id", "mail_id", "ip_id", "issuercountrycode", "txvariantcode", 
           "bin", "shoppercountrycode", "shopperinteraction", "cardverificationcodesupplied", 
           "cvcresponsecode", "accountcode", "amount", "currencycode", "label"]
df = df[columns]

[2019.05.08-01:16:17] Drop txid and bookingdate
[2019.05.08-01:16:17] Change simple_journal to label
[2019.05.08-01:16:17] Rearrange columns


In [13]:
# add new features

In [14]:
print(os.linesep + "Snippet of data after preprocessing")
df.head()


Snippet of data after preprocessing


,creationdate,card_id,mail_id,ip_id,issuercountrycode,txvariantcode,bin,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,accountcode,amount,currencycode,label
0,2015-07-01 00:00:41,90626,258032,222199,GB,visadebit,465944,GB,Ecommerce,True,1,UKAccount,6677.55795,GBP,0
1,2015-07-01 00:00:42,146288,75334,266345,GB,visadebit,492181,GB,Ecommerce,True,1,UKAccount,6415.43595,GBP,0
2,2015-07-01 00:00:48,43986,295389,288469,GB,visadebit,446238,GB,Ecommerce,True,1,UKAccount,6153.31395,GBP,0
3,2015-07-01 00:00:58,9184,19248,108767,GB,visadebit,475130,GB,Ecommerce,True,1,UKAccount,7332.86295,GBP,0
4,2015-07-01 00:01:11,261662,269209,329594,GB,visadebit,465902,GB,Ecommerce,True,1,UKAccount,9167.71695,GBP,0
